In [9]:
from dotenv import dotenv_values
import pandas as pd
from googleapiclient.discovery import build

In [10]:
# Đọc các giá trị từ .env
env_vars = dotenv_values('.env')

# Lấy giá trị API_KEY từ biến môi trường
api_key = env_vars.get('API_KEY')

# Channel ID
channel_id = "UCOmHUn--16B90oW2L6FRR3A" #BLACKPINK

In [11]:
def get_video_ids(api_key, channel_id, max_results=None):
    youtube = build('youtube', 'v3', developerKey=api_key)
    video_ids = []

    # Nếu không có số lượng được chỉ định, lấy 50 video
    if max_results is None:
        max_results = 50

    next_page_token = None
    while True:
        request = youtube.search().list(
            part='id',
            channelId=channel_id,
            maxResults=min(50, max_results),
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            if item['id']['kind'] == 'youtube#video':
                video_ids.append(item['id']['videoId'])

        if 'nextPageToken' in response:
            next_page_token = response['nextPageToken']
        else:
            break

        # Kiểm tra xem đã lấy đủ số lượng video cần thiết hay chưa
        if len(video_ids) >= max_results:
            break

    return video_ids[:max_results]

In [12]:
def get_video_info(api_key, video_ids):
    # Tạo DataFrame để lưu thông tin
    columns = ['Video_ID', 'Channel_ID', 'Publish_Date', \
               'Video_Title', 'Description', 'View_Count', \
                'Like_Count', 'Share_Count', 'Comment_Count']
    df = pd.DataFrame(columns=columns)

    # Tạo kết nối đến YouTube Data API với key và version
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Lấy thông tin cho từng video ID trong danh sách và thêm vào DataFrame
    for i, video_id in enumerate(video_ids):
        video_request = youtube.videos().list(
            part='snippet, statistics',
            id=video_id
        )
        video_response = video_request.execute()

        # Lấy thông tin của video
        if 'items' in video_response:
            for item in video_response['items']:
                video_title = item['snippet']['title']
                channel_id = item['snippet']['channelId']
                publish_date = item['snippet']['publishedAt']
                description = item['snippet']['description']
                
                # Kiểm tra xem có thông tin 'statistics' hay không
                if 'statistics' in item:
                    like_count = item['statistics'].get('likeCount', 'N/A')
                    share_count = item['statistics'].get('shareCount', 'N/A')
                    comment_count = item['statistics'].get('commentCount', 'N/A')
                    view_count = item['statistics'].get('viewCount', 'N/A')
                else:
                    like_count = 'N/A'
                    share_count = 'N/A'
                    comment_count = 'N/A'
                    view_count = 'N/A'

                # Nhập dữ liệu vào DataFrame
                row = [video_id, channel_id, publish_date, video_title, description, view_count, like_count, share_count, comment_count]
                df.loc[i] = row
        else:
            print(f"Video ID {video_id} không tồn tại hoặc không có thông tin.")

    return df


In [13]:
video_ids = get_video_ids(api_key, channel_id, 50)


In [14]:
df = get_video_info(api_key, video_ids)

In [15]:
df.head()

,Video_ID,Channel_ID,Publish_Date,Video_Title,Description,View_Count,Like_Count,Share_Count,Comment_Count
0,3fg8pyLKvXE,UCOmHUn--16B90oW2L6FRR3A,2016-07-06T01:00:06Z,BLACKPINK - DANCE PRACTICE VIDEO,CHOREOGRAPHY BY : PARRIS GOEBEL\n\n#YG #BLACKP...,88537909,2932163,N/A,90705
1,klbdGv5Bz3U,UCOmHUn--16B90oW2L6FRR3A,2023-04-07T14:21:36Z,붉게 타버려진 #지수 와 #영지 🌹 #JISOO_FLOWER #지수꽃 #ME #JI...,#JISOO_FLOWER #지수꽃 #ME #JISOO #BLACKPINK #블랙핑크...,73028572,3971659,N/A,24199
2,nzLq6D72PKU,UCOmHUn--16B90oW2L6FRR3A,2023-04-06T04:00:32Z,난 하얀 꽃잎처럼 날아가🤍 #JISOO_FLOWER #지수꽃 #ME #JISOO #...,#JISOO_FLOWER #지수꽃 #ME #JISOO #지수 #BLACKPINK #...,59857790,2626880,N/A,15422
3,UyruJZ9YrjE,UCOmHUn--16B90oW2L6FRR3A,2023-04-03T04:00:04Z,Dance with #ME 🌺 #JISOO_FLOWER #지수꽃 #JISOO #지수...,#JISOO_FLOWER #지수꽃 #JISOO #지수 #BLACKPINK #블랙핑크...,52338931,2970825,N/A,18514
4,LLWZOF7DiVg,UCOmHUn--16B90oW2L6FRR3A,2023-03-28T15:00:21Z,JISOO - ‘꽃(FLOWER)’ M/V TEASER,#JISOO #지수 #BLACKPINK #블랙핑크 #FIRSTSINGLEALBUM ...,18857466,1804536,N/A,127273


In [16]:
df.to_csv("data.csv", index=False)